Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from yeast8model import Yeast8Model

# Initialise model

In [ ]:
glc_exch_rate = 16.89
wt_ec = Yeast8Model("./models/ecYeastGEM_batch_8-6-0.xml")
wt_ec.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt_ec.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

# Ablate and store fluxes in each round

In [ ]:
wt_ec.ablation_result = wt_ec.ablate()

ablation_fluxes = wt_ec.ablation_fluxes
ablation_fluxes_diff = ablation_fluxes.copy()
ablation_fluxes_diff.pop("original")
for biomass_component, fluxes in ablation_fluxes_diff.items():
    ablation_fluxes_diff[biomass_component] = (
        ablation_fluxes[biomass_component] - ablation_fluxes["original"]
    )
    print(f"{biomass_component}")
    print(f"min {1e5 * ablation_fluxes_diff[biomass_component].min()} * 1e-5")
    print(f"max {1e5 * ablation_fluxes_diff[biomass_component].max()} * 1e-5")

Work

In [ ]:
from script_abl_vs_enz_use import (
    subsystem_dict,
    extract_protein_ids,
    get_participating_rxn_ids,
    get_subsystem_list,
)

In [ ]:
subsystem_dict

In [ ]:
s = ablation_fluxes_diff['lipid'].copy()

In [ ]:
enz_metabolite_ids = extract_protein_ids(s)
participating_rxn_ids, enz_usage_fluxes = get_participating_rxn_ids(
    enz_metabolite_ids, s, wt_ec
)
subsystem_list = get_subsystem_list(participating_rxn_ids, subsystem_dict)

In [ ]:
subsystem_list

In [ ]:
participating_rxn_df = pd.DataFrame(
    {
        "participating_rxn_id": participating_rxn_ids,
        "subsystem": subsystem_list,
        "enz_usage_flux": enz_usage_fluxes,
    }
)

In [ ]:
participating_rxn_df = participating_rxn_df[participating_rxn_df.subsystem != "Enzyme usage"]

In [ ]:
participating_rxn_df = participating_rxn_df.sort_values(by=['subsystem'])

In [ ]:
fluxes_array = participating_rxn_df.enz_usage_flux.to_numpy()
fluxes_array = np.reshape(fluxes_array, (-1,len(fluxes_array)))

In [ ]:
subsystem_list_sorted = participating_rxn_df.subsystem.to_list()

In [ ]:
# TODO: use this without crashing
subsystem_labels = []
compare_string = ''
for subsystem_string in subsystem_list_sorted:
    print(f'compare_string: {compare_string}')
    print(f'subsystem_string: {subsystem_string}')
    if subsystem_string == compare_string:
        subsystem_string = ' '
    else:
        compare_string = subsystem_string
    print(f'subsystem_string after: {subsystem_string}')
    subsystem_labels.append(subsystem_string)

In [ ]:
# 1d proof of concept
sns.heatmap(fluxes_array, center=0, robust=True, cmap='RdBu')

Loop

In [ ]:
list_participating_rxn_df = []

for idx, (biomass_component, fluxes) in enumerate(ablation_fluxes_diff.items()):
    s = fluxes.copy()
    
    enz_metabolite_ids = extract_protein_ids(s)
    participating_rxn_ids, enz_usage_fluxes = get_participating_rxn_ids(
        enz_metabolite_ids, s, wt_ec
    )
    subsystem_list = get_subsystem_list(participating_rxn_ids, subsystem_dict)
    
    enz_usage_flux_column = "enz_usage_flux_" + biomass_component
    participating_rxn_df = pd.DataFrame(
        {
            "participating_rxn_id": participating_rxn_ids,
            "subsystem": subsystem_list,
            enz_usage_flux_column: enz_usage_fluxes,
        }
    )
    list_participating_rxn_df.append(participating_rxn_df)

In [ ]:
left_columns = list_participating_rxn_df[0].iloc[:,0:2]

In [ ]:
enz_usage_columns = pd.concat(
    [list_participating_rxn_df[idx].iloc[:,-1]
     for idx in range(len(list_participating_rxn_df))
    ], axis=1
)

In [ ]:
all_fluxes_df = pd.concat([left_columns, enz_usage_columns], axis=1)

In [ ]:
all_fluxes_df = all_fluxes_df[all_fluxes_df.subsystem != "Enzyme usage"]

In [ ]:
all_fluxes_df = all_fluxes_df.sort_values(by=['subsystem'])

In [ ]:
# 2d proof of concept
fig, ax = plt.subplots(figsize=(10,50))
sns.heatmap(all_fluxes_df.iloc[:,2:], center=0, robust=True, cmap='RdBu')